# Prompt Engineering Lifecycle

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Log a trace

In [2]:
from app import langsmith_rag

question = "How fast can a cheetah run?"
langsmith_rag(question)

USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 197/197 [00:46<00:00,  4.23it/s]



"I don't know."

### Create a Dataset

Let's create a dataset with fascinating animal facts to evaluate this particular step of our application

In [3]:
from langsmith import Client

# Create dataset with random animal facts
example_dataset = [
    (
        "How fast can a cheetah run?",
        "Cheetahs are the fastest land animals in the world. They can reach speeds of up to 70 miles per hour (112 kilometers per hour) in short bursts covering distances up to 1,600 feet. However, they can only maintain this incredible speed for about 20-30 seconds before they need to rest.",
        "Cheetahs can run up to 70 mph (112 km/h) in short bursts for distances up to 1,600 feet. They are the fastest land animals but can only maintain top speed for 20-30 seconds."
    ),
    (
        "How many hearts does an octopus have?",
        "Octopuses have three hearts! Two of these hearts pump blood to the gills, while the third pumps blood to the rest of the body. Interestingly, the main heart stops beating when they swim, which is why they prefer crawling rather than swimming as it's less tiring for them.",
        "An octopus has three hearts - two pump blood to the gills and one pumps blood to the rest of the body. The main heart stops when swimming, making crawling less exhausting."
    ),
    (
        "How long do elephants sleep?",
        "Elephants are among the animals that sleep the least in the animal kingdom. Wild elephants typically sleep only 2-4 hours per day, often in short naps while standing up. They can sleep lying down, but this usually happens for only a few hours during the night. Baby elephants need more sleep than adults.",
        "Elephants sleep only 2-4 hours per day, often in short standing naps. They occasionally lie down for a few hours at night, making them one of the least-sleeping animals."
    ),
    (
        "Can penguins fly?",
        "While penguins cannot fly in the air, they are exceptional swimmers and can be considered to 'fly' underwater. Their wings have evolved into flippers that allow them to reach speeds of up to 22 mph underwater. Emperor penguins can dive to depths of over 1,800 feet and hold their breath for up to 22 minutes.",
        "Penguins cannot fly in air, but they 'fly' underwater using their wing-flippers to reach speeds of 22 mph. Emperor penguins can dive over 1,800 feet deep and hold their breath for 22 minutes."
    ),
    (
        "How long can a giraffe's tongue be?",
        "A giraffe's tongue can be up to 20 inches (50 centimeters) long! This incredibly long tongue helps them reach leaves high up in acacia trees. The tongue is also dark blue or black in color, which scientists believe helps protect it from sunburn since giraffes spend so much time with their tongues extended while feeding.",
        "A giraffe's tongue can reach up to 20 inches long and is dark blue or black in color. The length helps them reach high leaves, while the dark color protects against sunburn."
    ),
]

client = Client()
dataset_name = "Amazing Animal Facts"

# Create dataset
dataset = client.create_dataset(
    dataset_name=dataset_name, description="Fascinating questions and facts about animals from around the world"
)

# Prepare inputs and outputs
inputs = [{"question": q, "context": c} for q, c, _ in example_dataset]
outputs = [{"output": o} for _, _, o in example_dataset]

# Create examples in the dataset
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset.id,
)

{'example_ids': ['fda270cc-01a7-463d-921b-de0c2ff006ba',
  '1c19f41e-d80d-4a53-9830-7c54fb578244',
  'ecb8cbec-e602-43c9-8a0f-e41a1b8b37d1',
  '7bd60b72-2b34-43cb-89fc-04cba291b822',
  '4728929f-1224-437a-9b52-605f88755f3c'],
 'count': 5}

### Update our Application to use Prompt Hub

We're going to pretty much define the same RAG application as before - with one crucial improvement.

Instead of pulling our `RAG_PROMPT` from utils.py, we're going to connect to the Prompt Hub in LangSmith.

Let's add the code snippet that will pull down our prompt that we just iterated on!

In [4]:
from langsmith import Client
import os

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("animal-fun-fax:702d1c71", include_model=True)

In [8]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from langsmith.client import convert_prompt_to_openai_format
from openai import OpenAI
from typing import List
import nest_asyncio

MODEL_NAME = "gpt-4o-mini"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0

# TODO: Remove this hard-coded prompt and replace it with Prompt Hub
# RAG_SYSTEM_PROMPT = """You are an enthusiastic animal facts expert! 
# Use the following pieces of retrieved context to answer questions about animals and wildlife. 
# If you don't know the answer, just say that you don't know. 
# Provide fascinating and accurate information about animals in a fun, engaging way.
# Keep your answers concise but informative, using three sentences maximum.
# """

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's animal question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response about animals after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    
    # Method 1: Use the prompt directly since it's already bound to a model
    try:
        result = prompt.invoke({"context": formatted_docs, "question": question})
        return result
    except:
        # Method 2: Manual message formatting if prompt hub isn't working
        # Get the raw prompt template (without the model binding)
        raw_prompt_template = prompt.first
        raw_messages = raw_prompt_template.format_messages(context=formatted_docs, question=question)
        
        # Convert messages with proper role mapping
        def convert_role(msg_type):
            role_mapping = {
                'system': 'system',
                'human': 'user',
                'ai': 'assistant',
                'assistant': 'assistant'
            }
            return role_mapping.get(msg_type, msg_type)
        
        messages = [{"role": convert_role(msg.type), "content": msg.content} for msg in raw_messages]
        return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI for animal facts
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch animal-related documents
- Calls `generate_response` to generate a response about animals based on the fetched documents
- Returns the model response with fascinating animal facts
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    
    # Handle different response types
    if hasattr(response, 'choices'):
        return response.choices[0].message.content
    else:
        return str(response)

In [9]:
question = "How many hearts does an octopus have?"
langsmith_rag(question)

"content='An octopus has three hearts. Two pump blood to the gills, while the third pumps it to the rest of the body.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 336, 'total_tokens': 363, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CQFURLzPe2di2p6MAVEs6QEcTlmPz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--775651a7-e38a-477a-99e4-5f5e7745240d-0' usage_metadata={'input_tokens': 336, 'output_tokens': 27, 'total_tokens': 363, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}"